In [1]:
%%time
from osmgt import OsmGt


# build the graph
osm_data = OsmGt("Roanne")


ModuleNotFoundError: No module named 'graph_tool'

In [2]:
%%time
# to a graph from graph-tool
graph = osm_data.get_graph_from_location()



NameError: name 'osm_data' is not defined

In [ ]:
%%time
# to generate a svg graph output
from graph_tool.all import graph_draw 
from graph_tool.draw import sfdp_layout 

pos = sfdp_layout(graph)
graph_draw(graph, output="output.svg")

In [3]:
%%time
# to an numpy array
array = osm_data.get_road_numpy_array_from_location()



NameError: name 'osm_data' is not defined

In [4]:
%%time
# to a geodataframe
output_gdf = osm_data.get_road_gdf_from_location()


NameError: name 'osm_data' is not defined